# TAS4K8s App Example

* push an app - docker and source code based
* deploy minibroker
* create services
* bind services
* explore app

In [ ]:
#minikube environment
export TAS_ROOT=~/Downloads/tas-k8s-beta-workshop/tanzu-application-service
export TAS_DOMAIN=vcap.me
kubectl config use-context minikube

In [ ]:
#TKG standalone environment
export TAS_ROOT=~/Downloads/tas-k8s-beta02-workshop-tkg/tanzu-application-service
export TAS_DOMAIN=tas-tkg.lan
kubectl config use-context tkg-guest-admin@tkg-guest

In [ ]:
#common env
export WORKSPACE_ROOT=~/workspace
cd $WORKSPACE_ROOT

In [ ]:
kubectl config current-context

# Deploy App

In [ ]:
cf api --skip-ssl-validation https://api.$TAS_DOMAIN

In [ ]:
cd $TAS_ROOT
cf auth admin "$(bosh interpolate ../configuration-values/deployment-values.yml --path /cf_admin_password)"

In [ ]:
#needed in beta for now
cf enable-feature-flag diego_docker

In [ ]:
cf create-org test-org
cf create-space -o test-org test-space
cf target -o test-org -s test-space

In [ ]:
#push sample go app
cd $WORKSPACE_ROOT/test-app
cat manifest.yml

In [ ]:
cf push

In [ ]:
#also visible from kubectl logs
cf logs test-app

In [ ]:
#observe replicaset as scaling happens
#look at test app url /env to see load balancing
#use /exit to crash an instance
cf scale test-app -i 2

In [ ]:
cf set-env test-app TEST_ENV_VAR TEST_VALUE

In [ ]:
cf restart test-app

In [ ]:
cf env test-app

## Service Broker with Minibroker

In [ ]:
kubectl create ns minibroker

In [ ]:
helm repo add minibroker https://minibroker.blob.core.windows.net/charts

In [ ]:
helm repo update

In [ ]:
#install into desired namespace. not using service catalog because we will use TAS marketplace
helm install minibroker --namespace minibroker minibroker/minibroker \
  --set "deployServiceCatalog=false" --set "defaultNamespace=minibroker"

In [ ]:
kubectl get all --namespace minibroker

In [ ]:
cf create-service-broker minibroker user pass http://minibroker-minibroker.minibroker.svc.cluster.local

In [ ]:
cf service-brokers

In [ ]:
#see all the services the broker provides
cf service-access

In [ ]:
cf enable-service-access redis
cf enable-service-access mongodb
cf enable-service-access mariadb

In [ ]:
cf marketplace

In [ ]:
cf create-service redis 5-0-7 redis-svc

In [ ]:
cf create-service mariadb 10-3-22 mariadb-svc -c '{"db": {"name": "my_database"}}'

In [ ]:
cf services

In [ ]:
kubectl get pods --namespace minibroker

In [ ]:
kubectl get svc --namespace minibroker

### Test mariadb service
Edit mariadb service - change from ClusterIP to NodePort

```
kubectl port-forward <pod name of master> 3306:3306 --namespace minibroker &
```

In [ ]:
cf create-service-key mariadb-svc key

In [ ]:
cf service-key mariadb-svc key

### Connect to MariaDB through the port forward on your host

```mysql -h localhost --protocol=TCP -u root -p<password from service key>```

### Bind service to test-app

In [ ]:
cf bind-service test-app mariadb-svc

In [ ]:
cf restart test-app

In [ ]:
cf app test-app

# Tear down environment

In [ ]:
cf unbind-service test-app mariadb-svc

In [ ]:
cf delete test-app -f

In [ ]:
cf delete-service-key mariadb-svc key -f

In [ ]:
cf delete-service mariadb-svc -f

In [ ]:
cf services

In [ ]:
cf delete-service redis-svc -f

In [ ]:
cf delete-service-broker minibroker -f

In [ ]:
#kubectl delete namespace minibroker
helm uninstall minibroker --namespace minibroker

In [ ]:
kubectl delete namespace minibroker